# Descriptors and Abstract Base Classes

**Chapter 4 - Learning Python, 5th Edition**

This notebook explores two powerful mechanisms at the heart of Python's object model: the **descriptor protocol** and **abstract base classes (ABCs)**. Descriptors control how attribute access works under the hood and power `@property`, `@classmethod`, and `@staticmethod`. ABCs let you define formal interfaces that subclasses must implement. Together, they enable building robust, validated, and extensible class hierarchies.

## Section 1: How Attribute Lookup Works

When you access `obj.attr`, Python follows a well-defined lookup chain. Understanding this chain is essential before diving into descriptors.

**Lookup order:**
1. **Data descriptors** on the class (and its bases) — checked first via `type(obj).__mro__`
2. **Instance `__dict__`** — `obj.__dict__['attr']`
3. **Non-data descriptors and other class attributes** — `type(obj).__dict__['attr']` up the MRO
4. If nothing is found, `__getattr__` is called (if defined)

The orchestrator of this entire process is `__getattribute__`.

In [ ]:
# Demonstrating the __dict__ lookup chain
class Base:
    base_attr: str = "from Base"

class Parent(Base):
    parent_attr: str = "from Parent"

class Child(Parent):
    child_attr: str = "from Child"

obj: Child = Child()
obj.instance_attr = "from instance"  # type: ignore[attr-defined]

# Instance __dict__ holds instance attributes only
print(f"obj.__dict__: {obj.__dict__}")
print(f"Child.__dict__ keys: {list(Child.__dict__.keys())}")
print(f"Parent.__dict__ keys: {list(Parent.__dict__.keys())}")
print(f"Base.__dict__ keys: {list(Base.__dict__.keys())}")

# Python walks the MRO to find attributes
print(f"\nobj.instance_attr: {obj.instance_attr}")
print(f"obj.child_attr: {obj.child_attr}")
print(f"obj.parent_attr: {obj.parent_attr}")
print(f"obj.base_attr: {obj.base_attr}")

print(f"\nMRO: {[cls.__name__ for cls in type(obj).__mro__]}")


# __getattribute__ orchestrates every attribute access
class Logged:
    x: int = 10

    def __getattribute__(self, name: str) -> object:
        print(f"  __getattribute__ called for {name!r}")
        return super().__getattribute__(name)

logged: Logged = Logged()
print(f"\nAccessing logged.x:")
val: int = logged.x  # type: ignore[assignment]
print(f"Value: {val}")

## Section 2: The Descriptor Protocol

A **descriptor** is any object that defines at least one of:
- `__get__(self, obj, objtype=None)` — called on attribute access
- `__set__(self, obj, value)` — called on attribute assignment
- `__delete__(self, obj)` — called on attribute deletion
- `__set_name__(self, owner, name)` — called at class creation time (Python 3.6+)

**Data descriptor**: defines `__set__` and/or `__delete__` (takes priority over instance `__dict__`).  
**Non-data descriptor**: defines only `__get__` (instance `__dict__` takes priority).

In [ ]:
# Data descriptor vs non-data descriptor
class DataDescriptor:
    """Defines __get__ and __set__ -> data descriptor."""

    def __set_name__(self, owner: type, name: str) -> None:
        self.name: str = name

    def __get__(self, obj: object, objtype: type | None = None) -> object:
        if obj is None:
            return self
        print(f"  DataDescriptor.__get__ for {self.name!r}")
        return obj.__dict__.get(f"_desc_{self.name}", "<not set>")

    def __set__(self, obj: object, value: object) -> None:
        print(f"  DataDescriptor.__set__ for {self.name!r} = {value!r}")
        obj.__dict__[f"_desc_{self.name}"] = value


class NonDataDescriptor:
    """Defines only __get__ -> non-data descriptor."""

    def __set_name__(self, owner: type, name: str) -> None:
        self.name: str = name

    def __get__(self, obj: object, objtype: type | None = None) -> object:
        if obj is None:
            return self
        print(f"  NonDataDescriptor.__get__ for {self.name!r}")
        return f"non-data value for {self.name}"


class MyClass:
    data: DataDescriptor = DataDescriptor()
    non_data: NonDataDescriptor = NonDataDescriptor()

obj: MyClass = MyClass()

# Data descriptor intercepts both get and set
print("--- Data descriptor ---")
obj.data = 42
print(f"obj.data = {obj.data}")

# Non-data descriptor can be shadowed by instance __dict__
print("\n--- Non-data descriptor ---")
print(f"obj.non_data = {obj.non_data}")

# Shadow the non-data descriptor via instance __dict__
obj.__dict__["non_data"] = "instance value"
print(f"After shadowing: obj.non_data = {obj.non_data}")

# Data descriptors CANNOT be shadowed
obj.__dict__["data"] = "attempted shadow"
print(f"\nAttempted shadow on data descriptor: obj.data = {obj.data}")
print(f"obj.__dict__: {obj.__dict__}")

## Section 3: Building Practical Descriptors

Descriptors shine when you need **reusable validation** across multiple classes. Instead of writing repetitive `@property` getters and setters, define the logic once in a descriptor class.

In [ ]:
from typing import Any


class TypeChecked:
    """Descriptor that enforces a specific type on assignment."""

    def __init__(self, expected_type: type) -> None:
        self.expected_type: type = expected_type
        self.name: str = ""

    def __set_name__(self, owner: type, name: str) -> None:
        self.name = name

    def __get__(self, obj: Any, objtype: type | None = None) -> Any:
        if obj is None:
            return self
        return obj.__dict__.get(self.name)

    def __set__(self, obj: Any, value: Any) -> None:
        if not isinstance(value, self.expected_type):
            raise TypeError(
                f"{self.name!r} must be {self.expected_type.__name__}, "
                f"got {type(value).__name__}"
            )
        obj.__dict__[self.name] = value


class RangeValidator:
    """Descriptor that enforces a numeric range on assignment."""

    def __init__(self, min_val: float, max_val: float) -> None:
        self.min_val: float = min_val
        self.max_val: float = max_val
        self.name: str = ""

    def __set_name__(self, owner: type, name: str) -> None:
        self.name = name

    def __get__(self, obj: Any, objtype: type | None = None) -> Any:
        if obj is None:
            return self
        return obj.__dict__.get(self.name)

    def __set__(self, obj: Any, value: float) -> None:
        if not (self.min_val <= value <= self.max_val):
            raise ValueError(
                f"{self.name!r} must be between {self.min_val} and {self.max_val}, "
                f"got {value}"
            )
        obj.__dict__[self.name] = value


class NonEmpty:
    """Descriptor that rejects empty strings."""

    def __init__(self) -> None:
        self.name: str = ""

    def __set_name__(self, owner: type, name: str) -> None:
        self.name = name

    def __get__(self, obj: Any, objtype: type | None = None) -> Any:
        if obj is None:
            return self
        return obj.__dict__.get(self.name)

    def __set__(self, obj: Any, value: str) -> None:
        if not isinstance(value, str) or not value.strip():
            raise ValueError(f"{self.name!r} must be a non-empty string")
        obj.__dict__[self.name] = value


# Reuse descriptors across multiple classes
class Product:
    name: str = NonEmpty()  # type: ignore[assignment]
    price: float = RangeValidator(0.01, 10_000.0)  # type: ignore[assignment]
    quantity: int = TypeChecked(int)  # type: ignore[assignment]

    def __init__(self, name: str, price: float, quantity: int) -> None:
        self.name = name
        self.price = price
        self.quantity = quantity

    def __repr__(self) -> str:
        return f"Product({self.name!r}, price={self.price}, qty={self.quantity})"


class Employee:
    name: str = NonEmpty()  # type: ignore[assignment]
    age: int = RangeValidator(18, 120)  # type: ignore[assignment]
    employee_id: str = TypeChecked(str)  # type: ignore[assignment]

    def __init__(self, name: str, age: int, employee_id: str) -> None:
        self.name = name
        self.age = age
        self.employee_id = employee_id

    def __repr__(self) -> str:
        return f"Employee({self.name!r}, age={self.age}, id={self.employee_id!r})"


# Valid instances
product: Product = Product("Widget", 9.99, 100)
employee: Employee = Employee("Alice", 30, "EMP-001")
print(f"product: {product}")
print(f"employee: {employee}")

# Test validation errors
for label, call in [
    ("Empty name", lambda: Product("", 9.99, 1)),
    ("Negative price", lambda: Product("X", -5.0, 1)),
    ("Wrong type for qty", lambda: Product("X", 9.99, "ten")),
    ("Underage employee", lambda: Employee("Bob", 15, "EMP-002")),
]:
    try:
        call()
    except (TypeError, ValueError) as e:
        print(f"{label}: {e}")

## Section 4: Descriptors Under the Hood

Python's built-in `@property`, `@classmethod`, and `@staticmethod` are all implemented as descriptors. Understanding their internals clarifies how descriptors work in practice.

In [ ]:
from typing import Callable


# Manual implementation of @property as a descriptor
class MyProperty:
    """Simplified recreation of the built-in property descriptor."""

    def __init__(
        self,
        fget: Callable | None = None,
        fset: Callable | None = None,
        fdel: Callable | None = None,
    ) -> None:
        self.fget = fget
        self.fset = fset
        self.fdel = fdel

    def __get__(self, obj: object, objtype: type | None = None) -> object:
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj: object, value: object) -> None:
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj: object) -> None:
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def setter(self, fset: Callable) -> "MyProperty":
        return MyProperty(self.fget, fset, self.fdel)


class Circle:
    def __init__(self, radius: float) -> None:
        self._radius: float = radius

    @MyProperty
    def radius(self) -> float:
        return self._radius

    @radius.setter
    def radius(self, value: float) -> None:
        if value <= 0:
            raise ValueError("Radius must be positive")
        self._radius = value


c: Circle = Circle(5.0)
print(f"c.radius = {c.radius}")
c.radius = 10.0
print(f"After setting: c.radius = {c.radius}")

try:
    c.radius = -1.0
except ValueError as e:
    print(f"Validation: {e}")


# How @staticmethod and @classmethod work as descriptors
class MyStaticMethod:
    """Simplified recreation of the built-in staticmethod descriptor."""

    def __init__(self, func: Callable) -> None:
        self.func: Callable = func

    def __get__(self, obj: object, objtype: type | None = None) -> Callable:
        return self.func  # No binding at all


class MyClassMethod:
    """Simplified recreation of the built-in classmethod descriptor."""

    def __init__(self, func: Callable) -> None:
        self.func: Callable = func

    def __get__(self, obj: object, objtype: type | None = None) -> Callable:
        if objtype is None:
            objtype = type(obj)
        def bound_method(*args: object, **kwargs: object) -> object:
            return self.func(objtype, *args, **kwargs)
        return bound_method


class Demo:
    @MyStaticMethod
    def static_greet(name: str) -> str:
        return f"Hello, {name} (static)"

    @MyClassMethod
    def class_greet(cls, name: str) -> str:
        return f"Hello, {name} from {cls.__name__} (classmethod)"


print(f"\n{Demo.static_greet('Alice')}")
print(f"{Demo.class_greet('Bob')}")
print(f"{Demo().class_greet('Charlie')}")

# Prove built-in decorators are descriptors
print(f"\nproperty has __get__: {hasattr(property, '__get__')}")
print(f"property has __set__: {hasattr(property, '__set__')}")
print(f"staticmethod has __get__: {hasattr(staticmethod, '__get__')}")
print(f"classmethod has __get__: {hasattr(classmethod, '__get__')}")

## Section 5: Abstract Base Classes (ABC)

ABCs define **interfaces** that subclasses must implement. Python will refuse to instantiate a class that has not implemented all `@abstractmethod` members.

Key tools from the `abc` module:
- `ABC` — convenience base class (equivalent to `metaclass=ABCMeta`)
- `@abstractmethod` — marks a method that subclasses must override

> **Note:** `@abstractproperty` is deprecated since Python 3.3. Use `@property` combined with `@abstractmethod` instead.

In [ ]:
from abc import ABC, abstractmethod
import math


class Shape(ABC):
    """Abstract base class defining the interface for all shapes."""

    @abstractmethod
    def area(self) -> float:
        """Calculate the area of the shape."""
        ...

    @abstractmethod
    def perimeter(self) -> float:
        """Calculate the perimeter of the shape."""
        ...

    # Concrete method that uses abstract methods (template method pattern)
    def describe(self) -> str:
        return (
            f"{type(self).__name__}: "
            f"area={self.area():.2f}, perimeter={self.perimeter():.2f}"
        )

    # The modern way to define abstract properties (not @abstractproperty)
    @property
    @abstractmethod
    def name(self) -> str:
        """The display name of this shape."""
        ...


class Circle(Shape):
    def __init__(self, radius: float) -> None:
        self._radius: float = radius

    @property
    def name(self) -> str:
        return "Circle"

    def area(self) -> float:
        return math.pi * self._radius ** 2

    def perimeter(self) -> float:
        return 2 * math.pi * self._radius


class Rectangle(Shape):
    def __init__(self, width: float, height: float) -> None:
        self._width: float = width
        self._height: float = height

    @property
    def name(self) -> str:
        return "Rectangle"

    def area(self) -> float:
        return self._width * self._height

    def perimeter(self) -> float:
        return 2 * (self._width + self._height)


# Cannot instantiate abstract class
try:
    shape: Shape = Shape()  # type: ignore[abstract]
except TypeError as e:
    print(f"Cannot instantiate ABC: {e}")

# Concrete subclasses work fine
shapes: list[Shape] = [Circle(5.0), Rectangle(4.0, 6.0)]
for shape in shapes:
    print(f"{shape.name}: {shape.describe()}")

# isinstance / issubclass work with ABCs
print(f"\nisinstance(Circle(1), Shape): {isinstance(Circle(1), Shape)}")
print(f"issubclass(Rectangle, Shape): {issubclass(Rectangle, Shape)}")


# Incomplete implementation is caught at instantiation time
class IncompleteShape(Shape):
    """Missing perimeter and name -- Python will refuse to instantiate."""

    def area(self) -> float:
        return 0.0

try:
    broken: IncompleteShape = IncompleteShape()  # type: ignore[abstract]
except TypeError as e:
    print(f"\nIncomplete implementation: {e}")

## Section 6: Custom ABCs with \_\_subclasshook\_\_

ABCs support **virtual subclasses** — classes that are considered subclasses without actually inheriting. This is done via:
- `register()` — explicitly register a class as a virtual subclass
- `__subclasshook__` — automatically recognize classes based on their structure (structural typing)

This is conceptually similar to `typing.Protocol`, but works at runtime with `isinstance`/`issubclass`.

In [ ]:
from abc import ABC, ABCMeta


# Virtual subclass via register()
class Serializable(ABC):
    """ABC for objects that can serialize to a dict."""

    @abstractmethod
    def to_dict(self) -> dict[str, object]:
        ...


class ExternalConfig:
    """A class we don't control (e.g. from a third-party library)."""

    def __init__(self, host: str, port: int) -> None:
        self.host: str = host
        self.port: int = port

    def to_dict(self) -> dict[str, object]:
        return {"host": self.host, "port": self.port}


# Register as virtual subclass (no inheritance needed)
Serializable.register(ExternalConfig)

config: ExternalConfig = ExternalConfig("localhost", 8080)
print(f"isinstance(config, Serializable): {isinstance(config, Serializable)}")
print(f"issubclass(ExternalConfig, Serializable): {issubclass(ExternalConfig, Serializable)}")
print(f"ExternalConfig MRO: {[c.__name__ for c in ExternalConfig.__mro__]}")
print("Note: Serializable is NOT in the MRO -- it is a virtual subclass\n")


# Structural typing via __subclasshook__
class Renderable(ABC):
    """Any class with a render() -> str method is considered Renderable."""

    @abstractmethod
    def render(self) -> str:
        ...

    @classmethod
    def __subclasshook__(cls, C: type) -> bool:
        if cls is Renderable:
            # Check if C has a 'render' method anywhere in its MRO
            if any("render" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented


class HtmlWidget:
    """This class has render() but does not inherit from Renderable."""

    def render(self) -> str:
        return "<div>Hello</div>"


class PlainText:
    """No render method."""

    def display(self) -> str:
        return "plain text"


widget: HtmlWidget = HtmlWidget()
plain: PlainText = PlainText()

print(f"isinstance(widget, Renderable): {isinstance(widget, Renderable)}")
print(f"isinstance(plain, Renderable): {isinstance(plain, Renderable)}")


# Comparison with typing.Protocol (static, compile-time structural typing)
from typing import Protocol, runtime_checkable


@runtime_checkable
class RenderableProtocol(Protocol):
    def render(self) -> str: ...


print(f"\nProtocol check: isinstance(widget, RenderableProtocol): {isinstance(widget, RenderableProtocol)}")
print(f"Protocol check: isinstance(plain, RenderableProtocol): {isinstance(plain, RenderableProtocol)}")
print("\nABC __subclasshook__: runtime structural typing with full MRO inspection")
print("Protocol: primarily for static type checkers (mypy), runtime checks are shallow")

## Section 7: Putting It Together — Validated Data Model

This section combines descriptors and ABCs to build a small **validated model framework**, similar in spirit to Pydantic or SQLAlchemy's declarative models. The ABC enforces that every model implements `validate()`, while descriptors handle field-level type and constraint checking.

In [ ]:
from abc import ABC, abstractmethod
from typing import Any


# --- Descriptor layer: reusable validated fields ---

class Field:
    """Base descriptor for validated model fields."""

    def __init__(self) -> None:
        self.name: str = ""

    def __set_name__(self, owner: type, name: str) -> None:
        self.name = name

    def __get__(self, obj: Any, objtype: type | None = None) -> Any:
        if obj is None:
            return self
        return obj.__dict__.get(self.name)

    def __set__(self, obj: Any, value: Any) -> None:
        self.validate_field(value)
        obj.__dict__[self.name] = value

    def validate_field(self, value: Any) -> None:
        """Override in subclasses to add validation."""
        pass


class StringField(Field):
    """A non-empty string field with optional max length."""

    def __init__(self, max_length: int | None = None) -> None:
        super().__init__()
        self.max_length: int | None = max_length

    def validate_field(self, value: Any) -> None:
        if not isinstance(value, str):
            raise TypeError(f"{self.name!r} must be a string, got {type(value).__name__}")
        if not value.strip():
            raise ValueError(f"{self.name!r} must not be empty")
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(
                f"{self.name!r} exceeds max length {self.max_length} "
                f"(got {len(value)})"
            )


class IntField(Field):
    """An integer field with optional min/max constraints."""

    def __init__(
        self, min_val: int | None = None, max_val: int | None = None
    ) -> None:
        super().__init__()
        self.min_val: int | None = min_val
        self.max_val: int | None = max_val

    def validate_field(self, value: Any) -> None:
        if not isinstance(value, int) or isinstance(value, bool):
            raise TypeError(f"{self.name!r} must be an int, got {type(value).__name__}")
        if self.min_val is not None and value < self.min_val:
            raise ValueError(f"{self.name!r} must be >= {self.min_val}, got {value}")
        if self.max_val is not None and value > self.max_val:
            raise ValueError(f"{self.name!r} must be <= {self.max_val}, got {value}")


class FloatField(Field):
    """A float field with optional min/max constraints."""

    def __init__(
        self, min_val: float | None = None, max_val: float | None = None
    ) -> None:
        super().__init__()
        self.min_val: float | None = min_val
        self.max_val: float | None = max_val

    def validate_field(self, value: Any) -> None:
        if not isinstance(value, (int, float)) or isinstance(value, bool):
            raise TypeError(f"{self.name!r} must be numeric, got {type(value).__name__}")
        if self.min_val is not None and value < self.min_val:
            raise ValueError(f"{self.name!r} must be >= {self.min_val}, got {value}")
        if self.max_val is not None and value > self.max_val:
            raise ValueError(f"{self.name!r} must be <= {self.max_val}, got {value}")


# --- ABC layer: enforcing model interface ---

class Model(ABC):
    """Abstract base for validated data models."""

    @abstractmethod
    def validate(self) -> None:
        """Run cross-field validation after field-level checks."""
        ...

    def __init_subclass__(cls, **kwargs: Any) -> None:
        super().__init_subclass__(**kwargs)
        # Ensure subclasses define at least one Field descriptor
        fields: list[str] = [
            name for name, val in vars(cls).items()
            if isinstance(val, Field)
        ]
        if not fields and cls.__name__ != "Model":
            raise TypeError(f"{cls.__name__} must define at least one Field")

    def get_fields(self) -> dict[str, Any]:
        """Return a dict of all field names and their current values."""
        field_names: list[str] = [
            name for name, val in type(self).__dict__.items()
            if isinstance(val, Field)
        ]
        return {name: getattr(self, name) for name in field_names}

    def __repr__(self) -> str:
        fields_str: str = ", ".join(
            f"{k}={v!r}" for k, v in self.get_fields().items()
        )
        return f"{type(self).__name__}({fields_str})"


# --- Concrete models ---

class User(Model):
    username = StringField(max_length=20)
    email = StringField(max_length=100)
    age = IntField(min_val=0, max_val=150)

    def __init__(self, username: str, email: str, age: int) -> None:
        self.username = username
        self.email = email
        self.age = age
        self.validate()

    def validate(self) -> None:
        if "@" not in self.email:
            raise ValueError(f"Invalid email: {self.email!r}")


class Transaction(Model):
    description = StringField(max_length=200)
    amount = FloatField(min_val=0.01)
    quantity = IntField(min_val=1)

    def __init__(self, description: str, amount: float, quantity: int) -> None:
        self.description = description
        self.amount = amount
        self.quantity = quantity
        self.validate()

    def validate(self) -> None:
        total: float = self.amount * self.quantity
        if total > 1_000_000:
            raise ValueError(
                f"Transaction total ${total:,.2f} exceeds $1,000,000 limit"
            )


# Successful creation
user: User = User("alice", "alice@example.com", 30)
txn: Transaction = Transaction("Bulk order", 49.99, 10)
print(f"{user}")
print(f"{txn}")
print(f"User fields: {user.get_fields()}")
print(f"Transaction fields: {txn.get_fields()}")

# Demonstrate validation at multiple levels
print("\n--- Validation errors ---")
test_cases: list[tuple[str, Any]] = [
    ("Field-level: wrong type", lambda: User(123, "a@b.com", 25)),
    ("Field-level: empty string", lambda: User("", "a@b.com", 25)),
    ("Field-level: exceeds max length", lambda: User("a" * 25, "a@b.com", 25)),
    ("Field-level: out of range", lambda: User("bob", "b@b.com", -5)),
    ("Cross-field: invalid email", lambda: User("carol", "not-an-email", 28)),
    ("Cross-field: transaction limit", lambda: Transaction("Big order", 500_000.0, 3)),
]

for label, factory in test_cases:
    try:
        factory()
    except (TypeError, ValueError) as e:
        print(f"  {label}: {e}")

## Summary

### Attribute Lookup
- Python resolves `obj.attr` through: **data descriptors** on the class → **instance `__dict__`** → **non-data descriptors / class attrs** → `__getattr__`
- `__getattribute__` orchestrates the entire process on every attribute access

### Descriptor Protocol
- Descriptors define `__get__`, `__set__`, `__delete__`, and optionally `__set_name__`
- **Data descriptors** (`__set__` or `__delete__`) take priority over instance `__dict__`
- **Non-data descriptors** (only `__get__`) can be shadowed by instance attributes
- `@property`, `@classmethod`, and `@staticmethod` are all descriptor implementations

### Practical Descriptors
- Build reusable validators (`TypeChecked`, `RangeValidator`, `NonEmpty`) once, use across many classes
- `__set_name__` eliminates the need to repeat field names in descriptor constructors

### Abstract Base Classes
- `ABC` + `@abstractmethod` enforce that subclasses implement required methods
- Use `@property` + `@abstractmethod` instead of deprecated `@abstractproperty`
- `register()` creates virtual subclasses without inheritance
- `__subclasshook__` enables structural typing (duck typing with `isinstance` support)

### Descriptors + ABCs Combined
- Descriptors handle **field-level** validation (type checking, range, non-empty)
- ABCs enforce a **model interface** (`validate()` for cross-field rules)
- Together they form the foundation for frameworks like Pydantic and SQLAlchemy